# Load data from xlsx and train word2vec model

Load data from xlsx files

In [1]:
from os import listdir
import xlrd

In [2]:
path = 'Датасет/'
filenames = listdir(path)

text = ''
for filename in filenames:
    fileloc = path + filename
    workbook = xlrd.open_workbook(fileloc)
    sheet = workbook.sheet_by_index(0)

    for rowx in range(sheet.nrows):
        text += sheet.row_values(rowx)[0] 

Delete links from the data and split it into sentences

In [3]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [4]:
import nltk
import string

russian tokenizer is not supported by nltk originally, can download pickle file from https://github.com/Mottl/ru_punkt

Lowercase and lemmatize word using pymorphy

In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(word):
    ana = morph.parse(word)[0]
    try:
        lemma = ana.normal_form + '_' + ana.tag.POS
    except TypeError:
        lemma = ana.normal_form + '_' + 'OTHER'
    return lemma

In [6]:
def numberic(word):
    numbers = list('1234567890')
    for num in numbers:
        if num in word:
            return True
    return False

In [7]:
def tokenize_ru(file_text):
    # firstly let's apply nltk tokenization
    tokens = word_tokenize(file_text)

    # let's delete punctuation symbols
    tokens = [i for i in tokens if (i not in string.punctuation)]

    # deleting stop_words
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...', 
                       '/p', 'p', '/strong', 'strong', '/li', 'li', '…'])
    tokens = [i for i in tokens if (i not in stop_words) and (not(numberic(i)))]

    # cleaning words
    tokens = [i.replace("«", "").replace("»", "").replace('”', '') for i in tokens]
    
    # lemmatizing
    tokens = [lemmatize(i) for i in tokens]

    return tokens

In [8]:
text = re.sub('\[.*?\]', ' ', text)
sentences = [tokenize_ru(sent) for sent in sent_tokenize(text, 'russian')]

What is length of our corpus in words?

In [9]:
l = 0
for s in sentences:
    l += len(s)
print(l)

567625


It's rather small for training word2vec model, but we can try.

In [10]:
import gensim, gensim.models
from gensim.models import KeyedVectors

model = gensim.models.Word2Vec(sentences, window=20, size=300)

c:\users\tatiana\appdata\local\programs\python\python35-32\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
import os

directory = ## YOUR WORKING DIRECTIRY HERE ##
path = directory + '/mymodel.bin'

if not os.path.exists(directory):
    os.makedirs(directory)

# word_vectors = model.wv

# word_vectors.save(path)

model.save(path)

# word_vectors = KeyedVectors.load(fname, mmap='r')

In [12]:
print(len(model.wv.vocab)-1)

5965
